In [ ]:
from glob import glob
import os
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import pandas as pd
import scipy.signal
import scipy.spatial.distance as dist
from tqdm import tqdm
from simulation import SimulationLongitudinal, LeaderInteraction, LeaderInteractionParameters, \
    IDMPlus, IDMParameters
from load_interactions import load_interaction, save_interaction
from stats import KDE, kde_from_file
%matplotlib inline

# Estimate probability density function

In [ ]:
files = glob(os.path.join("data", "8_interactions", "*.hdf5"))

In [ ]:
# Show example
y = load_interaction(files[0])['vehicle_front']
i = y.index[scipy.signal.find_peaks(-y['vel_x_savgol'], prominence=1)[0]]
plt.plot(y['vel_x_savgol'])
plt.plot(y.loc[i, 'vel_x_savgol'], 'r.', ms=20)
plt.xlabel("Time [s]")
plt.ylabel("Speed [m/s]")

In [ ]:
def get_pars(vel_acc):
    if np.max(np.abs(np.diff(vel_acc['vel_x']))) > 1.5:
        return np.zeros((0, 4))
    
    data = vel_acc.copy()
    i = data.index[scipy.signal.find_peaks(-data['vel_x_savgol'], prominence=1)[0]]
    data['endspeed'] = np.nan
    data['endtime'] = np.nan
    data.loc[i, 'endspeed'] = data.loc[i, 'vel_x_savgol']
    data.loc[i, 'endtime'] = i
    data = data.fillna(method='backfill')
    data = data.dropna()

    data['duration'] = (data['endtime'] - data.index) / 10
    data['vdiff'] = data['endspeed'] - data['vel_x_savgol']
    data['amean'] = data['vdiff'] / data['duration']
    data = data.drop(i)
    return data[['vel_x', 'acc_x_savgol', 'vdiff', 'amean']].values[::10]

In [ ]:
filename_kde = os.path.join("data", "6_kde", "NGSIM_lead_interaction.p")
overwrite = False
if overwrite or not os.path.exists(filename_kde):
    parameters = [get_pars(load_interaction(file)['vehicle_front']) for file in files]
    kde = KDE(np.concatenate(parameters), scaling=True)
    kde.clustering(kde._maxdist()*2)
    kde.compute_bandwidth()
    print("Bandwidth: {:.4f}".format(kde.bandwidth))
    kde.pickle(filename_kde)
else:
    kde = kde_from_file(filename_kde)

# Simulation

In [ ]:
def leader_parameters(**kwargs):
    return LeaderInteractionParameters(init_position=kwargs["gap"],
                                       init_speed=kwargs["v0_lead"],
                                       init_acceleration=kwargs["a0_lead"],
                                       speed_difference=kwargs["dv"],
                                       duration=kwargs["duration"])

In [ ]:
def follower_parameters(**kwargs):
    return IDMParameters(amin=kwargs["amin"],
                         speed=kwargs["v0_host"],
                         n_reaction=int(kwargs["tr"]*100),
                         init_speed=kwargs["v0_host"],
                         init_position=0)

In [ ]:
s = SimulationLongitudinal(LeaderInteraction(), leader_parameters, IDMPlus(), follower_parameters)
s.min_simulation_time = 2

In [ ]:
s.simulation(dict(gap=100, v0_lead=0, a0_lead=-3, dv=0, duration=5, v0_host=30, amin=-8, tr=1),
             plot=True)

# Multiple simulations

In [ ]:
def get_other_pars(**kwargs):
    # Get the speed difference and the mean acceleration from the KDE.
    while True:
        (kwargs["dv"], kwargs["amean"]), = kde.conditional_sample([0, 1], [kwargs["v0_lead"], 
                                                                           kwargs["a0_lead"]])
        if np.sign(kwargs["dv"]) == np.sign(kwargs["amean"]):
            break
    kwargs["duration"] = kwargs["dv"] / kwargs["amean"]
    
    # Get reaction time from a lognormal distribution with mean=.92, std=0.28
    kwargs["tr"] = np.random.lognormal(np.log(.92), .28)
    
    # Get the braking capacity from a truncated normal distribution
    while True:
        kwargs["amin"] = np.random.normal(-9.7, 1.3)
        if -12.7 < kwargs["amin"] < -4.2:
            break
    
    return kwargs

In [ ]:
def get_probability(**kwargs):
    min_sim = 10
    max_sim = 100
    results = np.zeros(max_sim)
    for i in range(max_sim):
        parameters = get_other_pars(**kwargs)
        results[i] = s.simulation(parameters)
        
        if i+1 >= min_sim:
            kde_result = KDE(results[:i+1], scaling=True)
            kde_result.compute_bandwidth()
            cdf_zero = kde_result.cdf(np.array([0.0]))[0]
            if np.sqrt(cdf_zero*(1-cdf_zero)/(i+1)) < 0.01:
                break
    return cdf_zero

In [ ]:
get_probability(v0_lead=10, a0_lead=0, gap=20, v0_host=10)

# Show probabilities depending on 1 variable

In [ ]:
pars = dict(a0_lead=0, v0_host=20)
v0_leads = [10, 15, 20]
gaps = np.linspace(1, 25, 25)
for v0_lead in v0_leads:
    pars["v0_lead"] = v0_lead
    results = [get_probability(gap=gap, **pars) for gap in gaps]
    plt.plot(gaps, results, label="$v_{0,\mathrm{lead}}$="+"{:.0f} km/h, ".format(pars["v0_lead"]*3.6))
plt.xlabel("Initial gap [m]")
plt.ylabel("Collision probability")
plt.title("$a_{0,\mathrm{lead}}$=" + "{:.0f} m/s$^2$, ".format(pars["a0_lead"]) +
          "$v_{0,\mathrm{host}}$=" + "{:.0f} km/h".format(pars["v0_host"]*3.6))
plt.legend()

# Create grid to evaluate the collision probability

The parameters are:

- `v0_lead`: The lead speed at $t=0$;
- `a0_lead`: The lead acceleration at $t=0$;
- `v0_host`: The host speed at $t=0$;
- `loggap`: The log of the initial distance between the host and the leader.

In [ ]:
def grid_pars(interaction):
    pars = pd.DataFrame(interaction["vehicle_front"][["vel_x_savgol", "acc_x_savgol"]].values,
                        columns=["v0_lead", "a0_lead"], index=interaction["vehicle_front"].index)
    pars["v0_host"] = interaction["vehicle_rear"]["vel_x_savgol"]
    pars["loggap"] = np.log(interaction["interaction"]["relative_distance"])
    return pars

In [ ]:
filename = os.path.join("data", "7_simulation_results", "prob_collision.csv")
overwrite = False
if overwrite or not os.path.exists(filename):
    parameters = np.concatenate([grid_pars(load_interaction(file)) for file in files])
    grid = parameters.copy()
    scaling = [2, .5, 2, .25]
    grid[:, 0] = np.clip(grid[:, 0], 0, 26)
    grid[:, 1] = np.clip(grid[:, 1], -5, 5)
    grid[:, 2] = np.clip(grid[:, 2], 0, 26)
    grid[:, 3] = np.clip(grid[:, 3], 0, 5)
    grid = np.round(grid / scaling)
    grid = np.unique(grid, axis=0)
    grid = grid * scaling
else:
    df = pd.read_csv(filename)
    grid = df[["v0_lead", "a0_lead", "v0_host", "loggap"]].values

# Evaluate collision probability for grid

In [ ]:
def get_probability_grid_pars(row):
    return get_probability(v0_lead=row[0], a0_lead=row[1],
                           v0_host=row[2], gap=np.exp(row[3]))

In [ ]:
if overwrite or not os.path.exists(filename):
    prob_collision = [get_probability_grid_pars(row) for row in tqdm(grid)]
    df = pd.DataFrame(grid, columns=("v0_lead", "a0_lead", "v0_host", "loggap"))
    df["prob_collision"] = prob_collision
    df.to_csv(filename)
else:
    prob_collision = df["prob_collision"].values

# Interpolate collision probability for an interaction

In [ ]:
scaling = np.std(grid, axis=0)
grid_scaled = grid / scaling

In [ ]:
def prob_interaction(interaction):
    scaled_parameters = grid_pars(interaction) / scaling
    sq_distance = dist.cdist(grid_scaled, scaled_parameters, metric='sqeuclidean')
    weights = np.exp(-sq_distance / 2 / (0.3**2))  # Bandwidth of .3
    probability = np.dot(prob_collision, weights) / np.sum(weights, axis=0)
    return probability

In [ ]:
for file in tqdm(files):
    interaction = load_interaction(file)
    interaction["interaction"]["prob_collision"] = prob_interaction(interaction)
    save_interaction(interaction)